In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score

import random

In [2]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


# data processing

In [3]:
# same as sklearn 
df = pd.read_csv("data/my_horizon_data_all.csv", dtype={"subject": str})


seq_features = [
    'r0', 'c0',
    'r1', 'c1',
    'r2', 'c2',
    'r3', 'c3',
]
static_features = ['gameLength', 'uc']

target = 'c4'
X = df[seq_features + static_features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# split based on original data frame
h1_mask = X_test['gameLength'] == 1
h6_mask = X_test['gameLength'] == 6

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("--- Feature Data (X) ---")
print(X.head())
print("\n--- Target Data (y) ---")
print(y.head())

--- Feature Data (X) ---
     r0   c0    r1   c1    r2   c2    r3   c3  gameLength  uc
0  42.0  0.0  45.0  1.0  42.0  1.0  18.0  1.0           6   1
1  67.0  0.0  57.0  1.0  56.0  0.0  50.0  1.0           6   0
2  37.0  1.0  48.0  0.0  23.0  0.0  39.0  1.0           6   0
3  58.0  1.0  51.0  0.0  28.0  0.0  47.0  1.0           1   0
4   4.0  1.0  30.0  0.0  11.0  1.0  37.0  0.0           1   0

--- Target Data (y) ---
0    0.0
1    0.0
2    0.0
3    1.0
4    0.0
Name: c4, dtype: float64


convert to tensor for pytorch

In [4]:
print(type(X_train))
print(type(y_train))

<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>


In [5]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32) # sklearn output float64, doesn't work with torch
X_test_tensor = torch.tensor(X_test, dtype=torch.float32) 

y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.long) # pandas series to tensor
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long)

In [6]:
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32, shuffle=False)

In [7]:
# get the scaled tensor for h1 h6
# convert pandas series -> numpy array -> torch BoolTensor
h1_mask_bool = torch.tensor(h1_mask, dtype=torch.bool)
h6_mask_bool = torch.tensor(h6_mask, dtype=torch.bool)

X_test_h1 = X_test_tensor[h1_mask_bool]
X_test_h6 = X_test_tensor[h6_mask_bool]

y_test_h1 = y_test_tensor[h1_mask_bool]
y_test_h6 = y_test_tensor[h6_mask_bool]

test_loader_h1 = DataLoader(TensorDataset(X_test_h1, y_test_h1), batch_size=32, shuffle=False)
test_loader_h6 = DataLoader(TensorDataset(X_test_h6, y_test_h6), batch_size=32, shuffle=False)

# Model

In [8]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.fc2 = nn.Linear(32, 16)
        self.output = nn.Linear(16, output_size)

        self.relu = nn.ReLU()
        self.gelu = nn.GELU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.gelu(x)
        x = self.output(x)
        return x

# train / eval

In [21]:
def train(model, train_loader, criterion, optimizer, device): 
    model.train()
    train_loss = 0.0
    correct = 0
    total = len(train_loader)
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = model(inputs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        preds = logits.argmax(dim=1)
        train_loss += loss.item() * labels.size(0)
        correct += (preds == labels).sum().item()

    avg_loss = train_loss / total
    accuracy = correct / total
    return accuracy, avg_loss


def test(model, test_loader, criterion, device):
    model.eval()
    correct = 0
    total_loss = 0
    total = len(test_loader.dataset)

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            logits = model(inputs)
            loss = criterion(logits, labels)
            preds = logits.argmax(dim=1)

            correct += (preds == labels).sum().item()
            total_loss += loss.item() * labels.size(0)
            
    accuracy = correct / total
    avg_loss = total_loss / total
    return accuracy, avg_loss

# running experiment

In [22]:
num_features = len(seq_features) + len(static_features)
num_classes = 2

# model = MLP(num_features, num_classes).to(device)
# print(model)

In [23]:
model_dict = {
    "MLP": MLP(num_features, num_classes).to(device),
}

In [24]:

def train_and_evaluate(model, train_loader, test_loaders, criterion, optimizer, device, epochs):
    test_loader, test_loader_h1, test_loader_h6 = test_loaders
    train_loss_prog, train_acc_prog = [], []

    test_loss_prog, test_acc_prog = [], []

    test_acc_h1_prog, test_loss_h1_prog = [], []
    test_acc_h6_prog, test_loss_h6_prog = [], []

    epochs_without_improvement = 0 # for early stopping
    best_loss = float('inf')
    PATIENCE = 5
    final_epoch = epochs

    for epoch in range(epochs):
        train_acc, train_loss = train(model, train_loader, criterion, optimizer, device)

        test_acc, test_loss = test(model, test_loader, criterion, device)
        test_acc_h1, test_loss_h1 = test(model, test_loader_h1, criterion, device)
        test_acc_h6, test_loss_h6 = test(model, test_loader_h6, criterion, device)
        if (epoch+1) % 10 == 0 or epoch == 0:
            print(f"Epoch {epoch+1}: Loss: {test_loss:.4f} | overall: {test_acc:.4f} | H1 {test_acc_h1:.4f} | H6 {test_acc_h6:.4f}")

        train_acc_prog.append(train_acc)
        train_loss_prog.append(train_loss)

        test_loss_prog.append(test_loss)
        test_acc_prog.append(test_acc)

        test_acc_h1_prog.append(test_acc_h1)
        test_loss_h1_prog.append(test_loss_h1)

        test_acc_h6_prog.append(test_acc_h6)
        test_loss_h6_prog.append(test_loss_h6)

        # early stopping
        if test_loss < best_loss:
            best_loss = test_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement > PATIENCE: 
            print(f"Early stopping triggered: epoch {epoch+1} best_loss {best_loss:.4f}")
            final_epoch = epoch+1
            break

    return {
        "train_loss_prog": train_loss_prog,
        "train_acc_prog": train_acc_prog,
        "test_loss_prog": test_loss_prog,
        "test_acc_prog": test_acc_prog,
        "test_acc_h1_prog": test_acc_h1_prog,
        "test_loss_h1_prog": test_loss_h1_prog,
        "test_acc_h6_prog": test_acc_h6_prog,
        "test_loss_h6_prog": test_loss_h6_prog,
        "final_epoch": final_epoch
    }

In [25]:
epochs = 100
test_loaders = (test_loader, test_loader_h1, test_loader_h6)
for model_name, model in model_dict.items():
    print(f"\nTraining model: {model_name}")
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    history = train_and_evaluate(model, train_loader, test_loaders, criterion, optimizer, device, epochs=epochs)
    model_dict[model_name] = {
        "model": model,
        **history
    }



Training model: MLP
Epoch 1: Loss: 0.5042 | overall: 0.7739 | H1 0.8143 | H6 0.7336
Epoch 10: Loss: 0.4954 | overall: 0.7747 | H1 0.8121 | H6 0.7374
Epoch 20: Loss: 0.4932 | overall: 0.7778 | H1 0.8160 | H6 0.7397
Epoch 30: Loss: 0.4929 | overall: 0.7783 | H1 0.8139 | H6 0.7428
Early stopping triggered: epoch 34 best_loss 0.4916


In [26]:
dump_history = {}

for model_name, content in model_dict.items():
    # content = {"model": <model_object>, "train_acc": ..., "test_acc": ..., ...}
    filtered = {k: v for k, v in content.items() if k != "model"}
    dump_history[model_name] = filtered
# store the outputs 
import json
with open("output_mlp.json", "w") as f:
    json.dump(dump_history, f)

In [ ]:
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# epochs = 20
# loss_prog = []
# acc_prog = []
# acc_h1_prog = []
# acc_h6_prog = []

# for epoch in range(epochs):
#     loss = train(model, train_loader, criterion, optimizer, device, epoch)
#     acc = test(model, test_loader, device)
#     acc_h1 = test(model, test_loader_h1, device)
#     acc_h6 = test(model, test_loader_h6, device)
#     print(f"Epoch {epoch+1}: Loss: {loss:.4f} | overall: {acc:.4f} | H1 {acc_h1:.4f} | H6 {acc_h6:.4f}")

#     loss_prog.append(loss)
#     acc_prog.append(acc)
#     acc_h1_prog.append(acc_h1)
#     acc_h6_prog.append(acc_h6)

Epoch 1: Loss: 0.6018 | overall: 0.7836 | H1 0.8192 | H6 0.7474
Epoch 2: Loss: 0.5000 | overall: 0.7823 | H1 0.8249 | H6 0.7390
Epoch 3: Loss: 0.4888 | overall: 0.7964 | H1 0.8347 | H6 0.7574
Epoch 4: Loss: 0.4830 | overall: 0.7927 | H1 0.8347 | H6 0.7500
Epoch 5: Loss: 0.4779 | overall: 0.7979 | H1 0.8425 | H6 0.7526
Epoch 6: Loss: 0.4760 | overall: 0.8000 | H1 0.8430 | H6 0.7563
Epoch 7: Loss: 0.4739 | overall: 0.7961 | H1 0.8399 | H6 0.7516
Epoch 8: Loss: 0.4714 | overall: 0.7974 | H1 0.8430 | H6 0.7511
Epoch 9: Loss: 0.4698 | overall: 0.7940 | H1 0.8373 | H6 0.7500
Epoch 10: Loss: 0.4687 | overall: 0.8029 | H1 0.8466 | H6 0.7584
Epoch 11: Loss: 0.4678 | overall: 0.7979 | H1 0.8404 | H6 0.7547
Epoch 12: Loss: 0.4673 | overall: 0.7932 | H1 0.8332 | H6 0.7526
Epoch 13: Loss: 0.4657 | overall: 0.7995 | H1 0.8461 | H6 0.7521
Epoch 14: Loss: 0.4650 | overall: 0.8055 | H1 0.8512 | H6 0.7589
Epoch 15: Loss: 0.4634 | overall: 0.7971 | H1 0.8435 | H6 0.7500
Epoch 16: Loss: 0.4627 | overall: 

In [ ]:
from utils.train_eval import train_and_evaluate